### Exploratory Data Analysis
My goal is to create a model that will be capable of predicting value of an apartment in Warsaw. \
I will start with cleaning and reformatting the data, then I will proceed to feature engineering so I can finally explore it. 

## Dataset documentation
- location: address of the apartment
- total_price: price of the apartment in PLN
- price_per_sqm: price per square meter of the apartment in PLN
- area: area of the apartment in square meters
- rooms: number of rooms in the apartment
- finished: status of the apartment, e.g. ready to live, building shell
- floor: on which floor is the apartment located
- outside: is there garden/terrace/balcony
- rent: monthly rent rate in PLN
- elevator: is there a working elevator
- built: when was the building built
- b_type: type of the building e.g. skyscraper, block of flats
- link: link to Otodom listing of this apartment

In [28]:
from geopy.geocoders import Nominatim
import pandas as pd
import matplotlib as plt
import seaborn as sns
import numpy as np

In [29]:
df = pd.read_csv('details.txt', delimiter=';')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31850 entries, 0 to 31849
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   location       31845 non-null  object 
 1   total_price    31849 non-null  object 
 2   price_per_sqm  29473 non-null  object 
 3   area           31849 non-null  object 
 4   rooms          31849 non-null  object 
 5   finished       6912 non-null   object 
 6   floor          31404 non-null  object 
 7   outside        23252 non-null  object 
 8   rent           15569 non-null  object 
 9   elevator       27957 non-null  object 
 10  built          24779 non-null  float64
 11  b_type         23639 non-null  object 
 12  link           31850 non-null  object 
dtypes: float64(1), object(12)
memory usage: 3.2+ MB


In [30]:
df.isna().sum()

location             5
total_price          1
price_per_sqm     2377
area                 1
rooms                1
finished         24938
floor              446
outside           8598
rent             16281
elevator          3893
built             7071
b_type            8211
link                 0
dtype: int64

`location`, `total_price`, `area` and `rooms` have very few NaN values. However in my dataset empty values are often replaced by a string, mostly different for all columns, so let's try to convert those strings into NaN's. Also let's format the data in columns into numbers if it's possible. 

### `total_price`

In [31]:
df['total_price'].head()

0     1 049 000 zł
1     1 099 000 zł
2    444 563,23 zł
3       430 000 zł
4    621 422,01 zł
Name: total_price, dtype: object

In [32]:
df['total_price'] = df['total_price'].replace('Zapytaj o cenę', np.nan)
df['total_price'] = df['total_price'].str.replace(' ', '').str.slice(stop=-2)
print(df['total_price'].value_counts().head())
print(f"Number of NaN's in total_price is {df['total_price'].isna().sum()}")

total_price
799000    416
899000    295
750000    290
699000    276
850000    272
Name: count, dtype: int64
Number of NaN's in total_price is 2377


### `price_per_sqm`

In [33]:
df['price_per_sqm'].head()

0    14 204 zł/m²
1    18 381 zł/m²
2    13 768 zł/m²
3    17 530 zł/m²
4    16 919 zł/m²
Name: price_per_sqm, dtype: object

In [34]:
df['price_per_sqm'] = df['price_per_sqm'].str.replace(' ', '').str.slice(stop=-5)
print(df['price_per_sqm'].head())
print(f"Number of NaN's in price_per_sqm is {df['price_per_sqm'].isna().sum()}")

0    14204
1    18381
2    13768
3    17530
4    16919
Name: price_per_sqm, dtype: object
Number of NaN's in price_per_sqm is 2377


### `area`

In [35]:
df['area'].head()

0    73,85 m²
1    59,79 m²
2    32,29 m²
3    24,53 m²
4    36,73 m²
Name: area, dtype: object

In [36]:
df['area'] = df['area'].str.slice(stop=-3).str.replace(' ', '')
print(df['area'].value_counts().head())
print(f"Number of NaN's in area is {df['area'].isna().sum()}")

area
48    326
47    322
50    288
38    261
49    239
Name: count, dtype: int64
Number of NaN's in area is 1


### `rooms`

In [37]:
df['rooms'].value_counts()

rooms
3                11277
2                11241
4                 4798
1                 2955
5                 1225
6                  221
7                   59
więcej niż 10       37
8                   20
9                   10
10                   6
Name: count, dtype: int64

There's few apartments with a number of room bigger than 10. Since I want to focus on fairly average apartments in Warsaw I will drop these values. 

In [38]:
df = df[df['rooms']!='więcej niż 10']
df['rooms'] = df['rooms'].str.replace(' ', '')
print(df['rooms'].value_counts())
print(f"Number of NaN's in rooms is {df['rooms'].isna().sum()}")

rooms
3     11277
2     11241
4      4798
1      2955
5      1225
6       221
7        59
8        20
9        10
10        6
Name: count, dtype: int64
Number of NaN's in rooms is 1


### `finished`

In [39]:
df['finished'].value_counts()

finished
do zamieszkania    3363
do wykończenia     1658
Zapytaj            1338
do remontu          541
Name: count, dtype: int64

In [40]:
df['finished'] = df['finished'].replace('Zapytaj', np.nan)
print(f"Number of NaN's in finished is {df['finished'].isna().sum()}")

Number of NaN's in finished is 26251


### `floor`

In [41]:
df['floor'].value_counts().sort_index(ascending=False).head(15)

floor
suterena/6        1
suterena/4        5
suterena/2        5
suterena         11
poddasze/7        1
poddasze/52       5
poddasze/4        4
poddasze/3        3
poddasze/10       1
parter/9         32
parter/8        149
parter/7        161
parter/6        306
parter/5        511
parter/4       1011
Name: count, dtype: int64

Now I will format the data into numerical values. 'suterena' is basically a basement, so I will set it to -1. 'poddasze' is the top floor so I will set it to the max value of this building. 'parter' is 0. Also I will move the '/x' part of values into a new column called `max_floor`.

In [42]:
df['floor'] = df['floor'].str.replace(' ', '')
df['floor'] = df['floor'].str.replace('suterena', '-1')
df['floor'] = df['floor'].str.replace('parter', '0')
df['max_floor'] = pd.Series(dtype=int)
# in some rows floor is: >x/y, so we will just replace it with average of x+1 and y
for index, row in df.iterrows():
    val = row['floor']
    if pd.notna(val):
        if 'poddasze' in val or val=='poddasze':
            val.replace('poddasze', val[-1])
        if '/' in val:
            f, mf = val.split('/')
            if '>' in f:
                f = f[1:]
                f = str((int(f)+1+int(mf))//2)
            df.at[index, 'max_floor'] = int(mf)
            df.at[index, 'floor'] = f
        else:
            if '>' in val:
                val = str(int(val[1:])+1)
            df.at[index, 'floor'] = val

df = df[df['floor']!='poddasze'] # for some reason we were left with 14 rows with value equal to poddasze, couldnt fix it so just dropped them since its a small number
print(df['floor'].value_counts().sort_index(ascending=False).head(15))
print(f"Number of NaN's in floor is {df['floor'].isna().sum()}")


floor
9      492
8      647
7      956
6     1274
5     1922
4     3273
31      63
3     4697
27       8
21       3
20      23
2     5576
19      21
18      18
17      51
Name: count, dtype: int64
Number of NaN's in floor is 441


### `max_floor`
This really doesn't need any work since I've just created it.

In [43]:
print(df['max_floor'].value_counts().head(15))
print(f"Number of NaN's in max_floor is {df['max_floor'].isna().sum()}")

max_floor
4.0     5778
3.0     5212
5.0     4034
2.0     2925
6.0     2803
10.0    2341
7.0     2161
8.0     1866
9.0      664
11.0     529
15.0     444
1.0      439
12.0     423
14.0     221
13.0     129
Name: count, dtype: int64
Number of NaN's in max_floor is 1261


### `outside`

In [44]:
print(df['outside'].unique())
print(df['outside'].value_counts())
print(f"Number of NaN's in outside is {df['outside'].isna().sum()}")


[nan 'balkon' 'taras' 'balkon, ogródek' 'ogródek' 'balkon, taras'
 'ogródek, taras' 'balkon, ogródek, taras' 'taras, balkon'
 'taras, ogródek' 'taras, balkon, ogródek' 'ogródek, balkon']
outside
balkon                    17135
balkon, taras              1407
taras                      1334
ogródek                    1323
ogródek, taras              859
balkon, ogródek, taras      453
balkon, ogródek             428
taras, ogródek              156
taras, balkon               111
ogródek, balkon              15
taras, balkon, ogródek        5
Name: count, dtype: int64
Number of NaN's in outside is 8573


I will split the `outside` column into three new ones: `terrace`, `balcony` and `garden` conatining 0/1 values. Then `outside` column will be removed.

In [45]:
df['balcony'] = pd.Series(dtype=int)
df['terrace'] = pd.Series(dtype=int)
df['garden'] = pd.Series(dtype=int)

for index, row in df.iterrows():
    val = row['outside']
    if pd.notna(val):
        df.at[index, 'balcony'] = int('balkon' in val)
        df.at[index, 'terrace'] = int('taras' in val)
        df.at[index, 'garden'] = int('ogródek' in val)

df = df.drop('outside', axis=1)

print(df['balcony'].value_counts())
print(f"Number of NaN's in balcony is {df['balcony'].isna().sum()}")
print(df['terrace'].value_counts())
print(f"Number of NaN's in terrace is {df['terrace'].isna().sum()}")
print(df['garden'].value_counts())
print(f"Number of NaN's in garden is {df['garden'].isna().sum()}")



balcony
1.0    19554
0.0     3672
Name: count, dtype: int64
Number of NaN's in balcony is 8573
terrace
0.0    18901
1.0     4325
Name: count, dtype: int64
Number of NaN's in terrace is 8573
garden
0.0    19987
1.0     3239
Name: count, dtype: int64
Number of NaN's in garden is 8573


### `rent`

In [46]:
df['rent'].value_counts().sort_index(ascending=False).head(15)

rent
999 zł           8
998 zł           5
996 zł           1
995 zł           8
990 zł          46
99 999,99 zł     1
989 zł           1
987 zł           2
985 zł           1
983 zł          11
981 zł           1
980 zł          60
978 zł           1
977 zł           3
976,45 zł        3
Name: count, dtype: int64

In [47]:
df['rent'] = df['rent'].str.replace(' zł', '')
df['rent'] = df['rent'].str.replace(' ', '')
df['rent'] = df['rent'].str.replace(',', '.') # floating point

# quick check if there are any unexpected (string/NaN) values but there are not
for index, row in df.iterrows():
    val = row['rent']
    if pd.notna(val):
        val = float(val)

print(f"Number of NaN's in rent is {df['rent'].isna().sum()}")


Number of NaN's in rent is 16254


### `elevator`

In [48]:
df['elevator'].unique()

array(['nie', 'tak', nan], dtype=object)

I will just convert 'tak'/'nie' into 1/0

In [49]:
df['elevator'] = df['elevator'].map({'tak':'1', 'nie':'0'})
print(df['elevator'].value_counts())
print(f"Number of NaN's in elevator is {df['elevator'].isna().sum()}")


elevator
1    17817
0    10089
Name: count, dtype: int64
Number of NaN's in elevator is 3893


### `built`

In [50]:
df['built'].value_counts().sort_index().head(15)

built
2.0        3
3.0        1
19.0      10
20.0       2
52.0       4
56.0       1
58.0       1
70.0       2
75.0       1
193.0      4
200.0      1
750.0      1
1056.0     3
1073.0     2
1505.0     2
Name: count, dtype: int64

In [51]:
for index, row in df.iterrows(): # check if there are any unexcpected values
    val = row['built']
    if pd.notna(val):
        val = float(val)
print(f"Number of NaN's in built is {df['built'].isna().sum()}")

Number of NaN's in built is 7064


### `b_type`

In [52]:
print(df['b_type'].value_counts())
print(f"Number of NaN's in b_type is {df['b_type'].isna().sum()}")


b_type
blok                12179
apartamentowiec      7829
kamienica            2891
dom wolnostojący      338
szeregowiec           323
plomba                 25
loft                    4
Name: count, dtype: int64
Number of NaN's in b_type is 8210


### `location`
For the `location` column I'd like to split it into 3 new columns: `street`, `district` and `subdistrict`, because data in `location` is too messy. The initial plan was to do the splitting using Geopy, but luckily the data is pretty detailed. I will also drop all NaN values in `location` column, because there is very few of them. 

In [53]:
df['location'].head()

0    ul. Wiarusów 10, Gocławek, Praga-Południe, War...
1    ul. Obrzeżna, Służewiec, Służewiec, Mokotów, W...
2                 Raków, Włochy, Warszawa, mazowieckie
3                Okęcie, Włochy, Warszawa, mazowieckie
4        Stary Imielin, Ursynów, Warszawa, mazowieckie
Name: location, dtype: object

In [62]:
df.dropna(subset=['location'], inplace=True)
df['street'] = pd.Series(dtype=str)
df['district'] = pd.Series(dtype=str)
df['subdistrict'] = pd.Series(dtype=str)

for index, row in df.iterrows():
    address = str(row['location']).split(',')
    # there are some anomalies in the data, for now lets handle some of them with multiple cases
    if 'ul.' in address[0] or 'al.' in address[0] or 'ego' in address[0] or 'Wał Miedz' in address[0] or 'polki' in address[0] or 'Warszawa'==address[0]:
        street = address[0][3:].title()
        subdistrict = address[1].title()
        district = address[2].title()
    else:
        street = ''
        subdistrict = address[0].title()
        district = address[1].title()
    df.at[index, 'street'] = street
    df.at[index, 'district'] = district
    df.at[index, 'subdistrict'] = subdistrict
    

In [67]:
df['subdistrict'].value_counts().sort_values().head(200)

subdistrict
Kalenicka                                          1
Oś. Górce                                          1
Bodycha                                            1
Syrokomlii                                         1
Przy Iwickiej/Park/Klimatyczne/Budynek Z Cegły     1
                                                  ..
Wilanów Niski                                     11
 Las Bielański                                    11
Wschodu Słońca                                    11
Plac Konesera 6                                   11
 Piastów                                          12
Name: count, Length: 200, dtype: int64

We can see that our address columns still need some work. I will extract all rows where those anomalies happen and either fix them or drop them depending on a case. 